# Assignment 4

In [19]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import Doc
nlp_core = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")
import json
from datasets import load_dataset

In [3]:
dataset = load_dataset("imvladikon/english_news_weak_ner", "entities")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'sent_num', 'sentence', 'doc_title', 'score', 'entity_type', 'entity_text', 'start_char', 'end_char', 'tokens', 'raw_tags', 'ner_tags'],
        num_rows: 3515149
    })
})


In [4]:
# Retrieve the entity label names from the dataset
labels = dataset["train"].features["ner_tags"].feature.names

# Function to convert numerical NER tags into human-readable labels
def decode_ner_tags(example):
    example["ner_labels"] = [labels[tag] for tag in example["ner_tags"]]
    return example

# Apply the function to all data splits
dataset = dataset.map(decode_ner_tags)

In [11]:
train_dataset = dataset["train"]
df = train_dataset.to_pandas()

In [15]:
# Count unique document IDs
num_documents = df["doc_id"].nunique()
print(f"Number of unique documents: {num_documents}")

Number of unique documents: 309897


In [37]:
import spacy
from spacy.tokens import Doc
from spacy import displacy

# Load a blank spaCy model
nlp = spacy.blank("en")

# Function to visualize Named Entities
def visualize_ner(example):
    tokens = example["tokens"]
    labels = example["ner_labels"]
    start_chars = example["start_char"]
    end_chars = example["end_char"]

    # Convert entity positions into spaCy's expected format
    ents = []
    for start, end, label in zip(start_chars, end_chars, labels):
        if label != "O":  # Ignore "O" (Outside) labels
            ents.append(nlp.make_doc(example["sentence"]).char_span(start, end, label=label))

    # Create a spaCy Doc object
    doc = nlp.make_doc(example["sentence"])

    # Assign named entities (filter out None values)
    doc.ents = [ent for ent in ents if ent is not None]

    # Render visualization in Jupyter Notebook
    displacy.render(doc, style="ent", jupyter=True)

# Display the first example with highlighted named entities
visualize_ner(dataset["train"][1])